In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm, trange
from DLAIUtils import Utils

import pandas as pd
import time
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())



In [3]:
df = pd.read_csv('data/all-the-news-3.csv', nrows=99)
df.head()

,date,year,month,day,author,title,article,url,section,publication
0,2016-12-09 18:31:00,2016,12.0,9,Lee Drutman,We should take concerns about the health of li...,"This post is part of Polyarchy, an independent...",https://www.vox.com/polyarchy/2016/12/9/138983...,NaN,Vox
1,2016-10-07 21:26:46,2016,10.0,7,Scott Davis,Colts GM Ryan Grigson says Andrew Luck's contr...,The Indianapolis Colts made Andrew Luck the h...,https://www.businessinsider.com/colts-gm-ryan-...,NaN,Business Insider
2,2018-01-26 00:00:00,2018,1.0,26,NaN,Trump denies report he ordered Mueller fired,"DAVOS, Switzerland (Reuters) - U.S. President ...",https://www.reuters.com/article/us-davos-meeti...,Davos,Reuters
3,2019-06-27 00:00:00,2019,6.0,27,NaN,France's Sarkozy reveals his 'Passions' but in...,PARIS (Reuters) - Former French president Nico...,https://www.reuters.com/article/france-politic...,World News,Reuters
4,2016-01-27 00:00:00,2016,1.0,27,NaN,Paris Hilton: Woman In Black For Uncle Monty's...,Paris Hilton arrived at LAX Wednesday dressed ...,https://www.tmz.com/2016/01/27/paris-hilton-mo...,NaN,TMZ


In [7]:
import openai
openai.api_key = os.environ['OPENAI_API_KEY']
openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
INDEX_NAME = f'dl-ai-{openai.api_key[-36:].lower().replace("_", "-")}'
pinecone = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(INDEX_NAME)

pinecone.create_index(name=INDEX_NAME,
                      dimension=1536,
                      metric='cosine',
                      spec=ServerlessSpec(cloud='aws', region='us-west-2'))

index = pinecone.Index(INDEX_NAME)

In [8]:
def get_embeddings(articles, model="text-embedding-ada-002"):
    return openai_client.embeddings.create(input=articles, model=model)

CHUNK_SIZE = 400
TOTAL_ROWS = 20000

progress_bar = tqdm(total=TOTAL_ROWS)
chunks = pd.read_csv("data/all-the-news-3.csv", chunksize = CHUNK_SIZE, nrows = TOTAL_ROWS)

chunk_num = 0
for chunk in chunks:
    titles = chunk['title'].tolist()
    embeddings = get_embeddings(titles)
    prepped = [{"id": str(chunk_num*CHUNK_SIZE+i), 
                "values": embeddings.data[i].embedding,
                "metadata": {"title": titles[i]},} for i in range(0, len(titles))]
    chunk_num = chunk_num + 1
    if len(prepped) >= 200:
        index.upsert(prepped)
        prepped = []
    progress_bar.update(len(chunk))

  0%|          | 0/20000 [00:00<?, ?it/s]

In [9]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 19600}},
 'total_vector_count': 19600}

In [10]:
def get_recommendations(pinecone_index, search_term, top_k=10):
    embed = get_embeddings([search_term]).data[0].embedding
    res = pinecone_index.query(vector=embed, top_k=top_k, include_metadata=True)
    return res

reco = get_recommendations(index, "obama")
for r in reco.matches:
    print(f"{r.score} : {r.metadata['title']}")

0.859339297 : Obama's been a much more effective communicator than he gives himself credit for
0.852432191 : Obama’s parting message is a warning for Donald Trump
0.85172075 : America will miss Barack Obama's decency
0.850436866 : Barack Obama Golfs in Tuscany
0.849836826 : Barack Obama just stepped off the sidelines to defend Obamacare
0.848499358 : President Obama has a new plan to fight the opioid epidemic
0.848059535 : Obama: if you were fine with big government until it served black people, rethink your biases
0.846890688 : Muhammad Ali -- Obama Praises The Greatest
0.845816553 : President Obama: Michelle & I Are Gonna Be Renters
0.845515847 : Prez Obama -- I Support Colin Kaepernick's National Anthem Protest (VIDEO)


In [11]:
if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
  pinecone.delete_index(name=INDEX_NAME)

pinecone.create_index(name=INDEX_NAME, dimension=1536, metric='cosine',
  spec=ServerlessSpec(cloud='aws', region='us-west-2'))
articles_index = pinecone.Index(INDEX_NAME)

In [12]:
def embed(embeddings, title, prepped, embed_num):
    for embedding in embeddings.data:
        prepped.append({"id": str(embed_num),
                        "values": embedding.embedding,
                        "metadata": {"title": title}})
        embed_num += 1
        if len(prepped) >= 100:
            articles_index.upsert(prepped)
            prepped.clear()
    return embed_num

In [18]:
embed_num = 0
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 20)
prepped = []
df = pd.read_csv("data/all-the-news-3.csv", nrows=1000)
articles_list = df['article'].tolist()
titles_list = df['title'].tolist()

for i in tqdm(range(0, len(articles_list))):
    art = articles_list[i]
    title = titles_list[i]
    if art is not None and isinstance(art, str):
        texts = text_splitter.split_text(art)
        embeddings = get_embeddings(texts)
        embed_num = embed(embeddings, title, prepped, embed_num)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [19]:
articles_index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11000}},
 'total_vector_count': 11000}

In [20]:
reco = get_recommendations(articles_index, "obama", top_k=100)

seen = {}
for r in reco.matches:
    title = r.metadata['title']
    if title not in seen:
        print(f"{r.score} : {title}")
        seen[title] = "."

0.842926621 : In separate speeches Obama, Biden, Carter target Trump policies
0.840898454 : 'Cut it out,' Obama told Putin on cyber attacks
0.838349283 : The social soapbox: Democrats will embrace visual platforms at the 2016 Convention
0.837420762 : Waffle House Hero James Shaw Jr. Says Trump's Call Was 'Lackluster' But Appreciated
0.833408237 : Obama-Trump meeting 'less awkward' than some might have expected: White House
0.831431866 : Government shutdown: are Democrats becoming more like Republicans?
0.826482296 : Medicare-for-all will be shaped by these 3 questions
0.826108158 : Do terrorist attacks make a Trump win more likely? Here’s what the research suggests.
0.825046778 : Virginia governor's election results: polls, live updates, and what to expect
0.824916124 : Obama's quiet push to ban LGBTQ discrimination in the workplace
0.824618101 : The game theory behind Mitch McConnell's Supreme Court strategy
0.824217737 : This is what Obamacare sabotage looks like
0.821320057 : Why Ob